In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys
# Configure plotting
plt.rcParams["figure.figsize"] = (6,4)
# Custom libraries and routines
!rm -rf ~/personal_gkyl_scripts/pygkyl/pygkyl.egg-info
!rm -rf ~/personal_gkyl_scripts/pygkyl/build
!{sys.executable} -m pip install ~/personal_gkyl_scripts/pygkyl > ~/personal_gkyl_scripts/pygkyl/install.log
import pygkyl
# Setup path to data
home_dir = os.path.expanduser("~")
repo_dir = home_dir+'/personal_gkyl_scripts/'
simdir = '/Users/ahoffman/gkeyll_main/gkeyll/rt_gk_core.o/'
fileprefix = 'rt_gk_core'
# Create simulation object
simulation = pygkyl.Simulation(dimensionality='3x2v')
simulation.set_phys_param(
    eps0 = 8.854e-12,       # Vacuum permittivity [F/m]
    eV = 1.602e-19,         # Elementary charge [C]
    mp = 1.673e-27,         # Proton mass [kg]
    me = 9.109e-31,         # Electron mass [kg]
)
def qprofile(R):
    a = [497.3420166252413, -1408.736172826569, 1331.4134861681464, -419.00692601227627]
    return a[0]*R**3 + a[1]*R**2 + a[2]*R + a[3]
simulation.set_geom_param(
    B_axis = 1.4,           # Magnetic field at magnetic axis [T]
    R_axis      = 0.8727315068,         # Magnetic axis major radius
    Z_axis      = 0.1414361745,         # Magnetic axis height
    R_LCFSmid   = 1.0968432365089495,   # Major radius of LCFS at the midplane
    a_shift     = 0.25,                 # Parameter in Shafranov shift
    kappa       = 1.5,                  # Elongation factor
    delta       = 0.3,                  # Triangularity factor
    qprofile_R  = qprofile,             # Safety factor
    x_LCFS      = 0.04,                 # position of the LCFS (= core domain width)
    x_out       = 0.08                  # SOL domain width
)
# Define the species
ion = pygkyl.Species(name='ion',
              m=2.01410177811*simulation.phys_param.mp, # Ion mass
              q=simulation.phys_param.eV,               # Ion charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
elc = pygkyl.Species(name='elc',
              m=simulation.phys_param.me, 
              q=-simulation.phys_param.eV, # Electron charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
# Add them to the simulation (we need to know this before setting up the data parameters)
simulation.add_species(ion)
simulation.add_species(elc)
# This call will set up the data structure of the simulation and set up a large dictionary 
# conaining the receipes of many post processing quantities, see simulation.data_param.info()
simulation.set_data_param( simdir = simdir, fileprefix = fileprefix, species = simulation.species)
# This is the first call that will load data. 

In [ ]:
# custom density profile
nmax = 7e19  # peak density
n_min = 1e19  # minimum density
R_transition = 0.9  # center of transition region
width = 20.0  # controls transition width
R_LCFS = simulation.geom_param.R_LCFSmid
R_axis = simulation.geom_param.R_axis
amid = simulation.geom_param.a_mid
r0 = simulation.geom_param.r0
def density_custom_profile(R):
    return n_min + (nmax - n_min) * 0.5 * (1 + np.tanh(width * (R_transition - R)))

def tanh_profile(R):
    x = R-R_axis-r0
    n0 = 5e19
    x0 = 0.01
    c1 = 0.5
    c2 = 8.0
    c3 = 0.005
    return n0 * (c1 * (1.0 + np.tanh(c2 * (-10 * (x + x0)))) + c3)

R = np.linspace(0.1, 1.3, 400) * amid + R_axis
# plot density profile
plt.plot((R-R_axis)/r0, density_custom_profile(R), label='Custom')
plt.plot((R-R_axis)/r0, tanh_profile(R), label='Tanh Fit', linestyle='--')
plt.xlabel('R [m]')
plt.ylabel('Density [1/m³]')
plt.title('Custom Density Profile')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Print some key values to verify
print(f"Density at R=0.25: {density_custom_profile(0.25):.2e} m⁻³")
print(f"Density at R=0.6: {density_custom_profile(0.6):.2e} m⁻³")
print(f"Density at R=0.8: {density_custom_profile(0.8):.2e} m⁻³")
print(f"Density at R=1.0: {density_custom_profile(1.0):.2e} m⁻³")

In [ ]:
# Function to fit analytical profiles to experimental data and generate C code
def fit_experimental_profiles(write_c_functions=False):
    """
    Fit tanh analytical profiles to experimental density and temperature data.
    
    Parameters:
    -----------
    write_c_functions : bool
        Whether to write C function definitions
    
    Returns:
    --------
    density_fit, temperature_fit, ne_params, Te_params
    """
    from scipy.optimize import minimize
    
    c_guess = 20 # Steepness of the tanh profile initial guess
    c_bound = (10, 100) # Steepness bounds

    # Load experimental data (R coordinates are directly in the files)
    exp_data_ne = np.loadtxt(repo_dir + 'studies/meta_analysis/exp_ne_PT_TS.txt', delimiter=',')
    exp_data_Te = np.loadtxt(repo_dir + 'studies/meta_analysis/exp_Te_PT_TS.txt', delimiter=',')
    
    rho_ne_exp, ne_exp = exp_data_ne[:, 0], exp_data_ne[:, 1]
    rho_Te_exp, Te_exp = exp_data_Te[:, 0], exp_data_Te[:, 1]
    R_ne_exp = R_axis + rho_ne_exp * amid
    R_Te_exp = R_axis + rho_Te_exp * amid

    nmax_guess = np.max(ne_exp)
    nmax_bound = (0.5*nmax_guess, 2*nmax_guess)
    nmin_guess = np.min(ne_exp)
    nmin_bound = (0.5*nmin_guess, 2*nmin_guess)
    Tmax_guess = np.max(Te_exp)
    Tmax_bound = (0.5*Tmax_guess, 2*Tmax_guess)
    Tmin_guess = np.min(Te_exp)
    Tmin_bound = (0.5*Tmin_guess, 2*Tmin_guess)
    R_min = np.min([R_ne_exp.min(), R_Te_exp.min()])
    R_max = np.max([R_ne_exp.max(), R_Te_exp.max()])
    R_guess = 0.5 * (R_min + R_max)
    R_bound = (R_min, R_max)
    
    # Define tanh profile functions for fitting
    def density_profile_fit(R, params):
        """Tanh-based density profile: nmax * (1 + tanh(c1 * (R0 - R))) + nmin"""
        nmax, nmin, R0, c1 = params
        return nmin + (nmax - nmin) * 0.5 * (1 + np.tanh(c1 * (R0 - R)))

    def temperature_profile_fit(R, params):
        """Tanh-based temperature profile: Tmax * (1 + tanh(c1 * (R0 - R))) + Tmin"""
        Tmax, Tmin, R0, c1 = params
        return Tmin + (Tmax - Tmin) * 0.5 * (1 + np.tanh(c1 * (R0 - R)))

    # Initial guesses and bounds for tanh profiles
    ne_guess = [nmax_guess, nmin_guess, R_guess, c_guess]
    ne_bounds = [nmax_bound, nmin_bound, R_bound, c_bound]
    Te_guess = [Tmax_guess, Tmin_guess, R_guess, c_guess]
    Te_bounds = [Tmax_bound, Tmin_bound, R_bound, c_bound]

    # Objective functions for optimization
    def objective_density(params):
        ne_model = density_profile_fit(R_ne_exp, params)
        return np.sum((ne_model - ne_exp)**2) / len(ne_exp)
    
    def objective_temperature(params):
        Te_model = temperature_profile_fit(R_Te_exp, params)
        return np.sum((Te_model - Te_exp)**2) / len(Te_exp)
    
    # Optimize density profile
    result_ne = minimize(objective_density, ne_guess, bounds=ne_bounds, method='L-BFGS-B')
    ne_params_opt = result_ne.x
    
    # Optimize temperature profile  
    result_Te = minimize(objective_temperature, Te_guess, bounds=Te_bounds, method='L-BFGS-B')
    Te_params_opt = result_Te.x
    
    # Create optimized profile functions
    def density_profile_optimized(R):
        return density_profile_fit(R, ne_params_opt)
    
    def temperature_profile_optimized(R):
        return temperature_profile_fit(R, Te_params_opt) * simulation.phys_param.eV
    
    # Plot comparison
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Density comparison
    R_plot = np.linspace(R.min(), R.max(), 200)
    ax1.scatter(R_ne_exp, ne_exp/1e19, color='red', label='Experimental data', s=30, alpha=0.7)
    ax1.plot(R_plot, density_profile_optimized(R_plot)/1e19, 'b-', linewidth=2, 
             label='Fitted tanh profile')
    ax1.set_xlabel('R [m]')
    ax1.set_ylabel('Density [1e19 m-3]')
    ax1.set_title('Density Profile Fit (tanh)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Temperature comparison
    ax2.scatter(R_Te_exp, Te_exp, color='red', label='Experimental data', s=30, alpha=0.7)
    ax2.plot(R_plot, temperature_profile_optimized(R_plot)/simulation.phys_param.eV, 'b-', linewidth=2, 
             label='Fitted tanh profile')
    ax2.set_xlabel('R [m]')
    ax2.set_ylabel('Temperature [eV]')
    ax2.set_title('Temperature Profile Fit (tanh)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"  Density RMS error = {np.sqrt(result_ne.fun):.2e}")
    print(f"  Temperature RMS error = {np.sqrt(result_Te.fun):.2f}")
    
    # Write C functions if requested
    if write_c_functions:
        # Write density C function
        print("\n// Density initial condition (fitted tanh profile)")
        print("void density_init(double t, const double * GKYL_RESTRICT xn, double* GKYL_RESTRICT fout, void *ctx)")
        print("{")
        print("  double x = xn[0], z = xn[1];")
        print("  struct gk_app_ctx *app = ctx;")
        print("  double R = app->Rmid_min + x;")
        print(f"  double nmax = {ne_params_opt[0]:.2e};")
        print(f"  double nmin = {ne_params_opt[1]:.2e};")
        print(f"  double R0 = {ne_params_opt[2]:.4f};")
        print(f"  double c1 = {ne_params_opt[3]:.3f};")
        print("  fout[0] = nmin + (nmax - nmin) * 0.5 * (1. + tanh(c1 * (R0 - R)));")
        print("}")
        
        # Write temperature C function
        print("\n// Temperature initial condition (fitted tanh profile)")
        print("void temp_init(double t, const double * GKYL_RESTRICT xn, double* GKYL_RESTRICT fout, void *ctx)")
        print("{")
        print("  double x = xn[0], z = xn[1];")
        print("  struct gk_app_ctx *app = ctx;")
        print("  double R = app->Rmid_min + x;")
        print(f"  double Tmax = {Te_params_opt[0]:.1f};")
        print(f"  double Tmin = {Te_params_opt[1]:.1f};")
        print(f"  double R0 = {Te_params_opt[2]:.4f};")
        print(f"  double c1 = {Te_params_opt[3]:.3f};")
        print("  fout[0] = Tmin + (Tmax - Tmin) * 0.5 * (1. + tanh(c1 * (R0 - R)));")
        print("}")
    
    return density_profile_optimized, temperature_profile_optimized, ne_params_opt, Te_params_opt

# Run the tanh fit
print("Fitting tanh profiles to experimental data...")
density_fit, temp_fit, ne_params, Te_params = fit_experimental_profiles(write_c_functions=True)
